# Decomposition of a general single-qutrit gate
> And a couple of words about Euler's and KAK decompositions
- toc: true 
- badges: true
- comments: true
- categories: [quantum concepts, compilation]
- image: images/relative_toffoli.png

In [7]:
#collapse

try:
  import qiskit
except ImportError:
  !pip install qiskit
try:
  import pylatexenc
except ImportError:
  !pip install pylatexenc
  
from qiskit import transpile, QuantumCircuit, QuantumRegister
from qiskit.circuit.library import *
from qiskit.quantum_info import OneQubitEulerDecomposer, random_clifford, Operator, Statevector
from qiskit.extensions import UnitaryGate
from qiskit.circuit import Instruction
from typing import Union
import numpy as np
from scipy.stats import unitary_group

# What this post is about

The main purpose of this post is to give an explicit formula for decompositions of an arbitrary single-qutrit gate. I'll also give ...b


# Euler's decomposition of an arbitrary single-qubit gate

## Elementary single-qubit rotations
Me (and probably you too) usually take for granted that an arbitrary single-qubit gate $U$ can be decomposed into a product of three elementary rotations. The rotations are usually chosen as

\begin{align}
&R_X(\theta)=e^{-i\theta X/2}=\begin{pmatrix} \cos\frac{\theta}2 & -i\sin\frac{\theta}2 \\  -i\sin\frac{\theta}2 & \cos\frac{\theta}2\end{pmatrix}\\
&R_Y(\theta)=e^{-i\theta Y/2}=\begin{pmatrix} \cos\frac{\theta}2 & -\sin\frac{\theta}2 \\  \sin\frac{\theta}2 & \cos\frac{\theta}2\end{pmatrix}\\
&R_Z(\theta)=e^{-i\theta Z/2}=\begin{pmatrix} e^{-i \theta/2} & 0 \\ 0 & e^{i\theta/2}\end{pmatrix}
\end{align}

and a sample decomposition reads

## Euler's decomposition
$$U = R_X(\theta_1)R_Z(\theta_2)R_X(\theta_3) \label{XZX}$$

Chosing angles $\theta_i$ appropriately one can obtain any $2\times 2$ special (with unit determinant) unitary matrix $U\in SU(2)$. 

## Making sense of Euler's decomposition
Let me make a couple of remarks in an attempt to make relation \eqref{XZX} a bit more intuitive. First of all, the dimension of the group $SU(n)$, i.e. a group of $n\times n$ unitary matrices with unit determinant is equal to

$$\operatorname{dim} SU(n)=n^2-1 \ .$$
For $n=2$, corresponding to a single-qubit gate, $\operatorname{dim}=3$. This implies that the right-hand side in eq.\eqref{XZX} needs to have at least three parameters and hence three elementary rotations. Good, this explains why are there three terms. (This technique of unsubtle parameter counting in fact extends far more generally. For example, it can be used to deduce a minumum amount of $CNOT$ gate necessary to compile an arbitrary $n-$qubit unitary, as I discussed [here](https://idnm.github.io/blog/machine%20learning/compilation/qiskit/paper%20review/2021/07/22/Machine-learning-compilation-of-quantum-circuits.html#Theoretical-lower-bound-and-quantum-Shannon-decomposition)).

## A word about Cartan decomposition (KAK)
Now, is there anything special about this particular combination of rotations $XZX$? Well, yes and no. In fact, almost any three single-parameter unitaries would work. Other possible decompositions are $XYX$, $YZY$ etc, but also less common options exist such as $XYZ$. 

However, decompositions like $XZX$ are special in a sense that they are examples of Cartan (aka KAK) decompositions, hinging on the structe of the underlying $su(2)$ algebra. Decompositions of two-qubit gates into $CNOT$ gates and single-qubit rotations is another important example where KAK arise. I will not cover the subject here, but can recommend a quite readable introduction with applications to quantum gates [https://arxiv.org/abs/quant-ph/0010100](https://arxiv.org/abs/quant-ph/0010100).


## Finding angles in Euler's decomposition

I have not told you yet how to find angles in the Euler decomposition, so here we go. The key observation is that because $XZ=-XZ$ we have
$$X R_Z(\theta)X=R_Z(-\theta),\quad X R_X(\theta)X=R_X(\theta) \ .$$
Multiplying \eqref{XZX} by $X$ from left and right we get
$$XUX=R_X(\theta_1)R_Z(-\theta_2)R_X(\theta_3) \ .$$
Therefore,
$$M:= UXU^\dagger X=R_x(\theta_1)R_Z(2\theta_2)R_x^\dagger(\theta_1)$$.

Since $R_Z$ is a diagonal gate, the right hand side is nothing else but a unitary diagonalization of matrix $M$. Therefore, finding $\theta_1$ and $\theta_2$ reduce to the standard diagonalization procedure. The last parmeter $\theta_3$ is then read off from $R_X(\theta_3)=R_Z^\dagger(\theta_2) R_X^\dagger(\theta_1)U$. 


Here is a quick and dirty code that implements the procedure. It won't handle some edge cases and may not be the most efficient, but I'll be happy with it's performance on random unitaries. 

In [59]:
# Define X, RX, and RZ.

X = np.array([[0, 1], [1, 0]])

def RX(theta):
    return np.array([[np.cos(theta/2), -1j*np.sin(theta/2)],[-1j*np.sin(theta/2), np.cos(theta/2)]])

def RZ(theta):
    return np.array([[np.exp(-1j*theta/2), 0],[0, np.exp(1j*theta/2)]])

# Decomposition routine.

def XZX_decomposition(U):
    
    assert np.allclose(U.conj().T @ U, np.eye(2)), 'Input matrix is not unitary.'
    assert np.allclose(np.linalg.det(U), 1), 'Input matrix has non-unit determinant.'
    
    
    # Construct the matrix to diagonalize.
    M = U @ X @ U.conj().T @ X
    
    # Diagonalization.
    RZ_squared, RX1 = np.linalg.eig(M)
    
    # Normalize RX1 to have a zero global phase.
    RX1 = RX1*np.abs(RX1[0, 0])/RX1[0, 0]
    
    # Determine theta_1 and theta_2 from diagonalization.
    theta_1 = 2*np.arcsin(1j*RX1[0, 1])
    theta_2 = 1j*np.log(RZ_squared[0])
    
    # Determine RX(theta_3) and theta_3. 
    RX2 = RZ(-theta_2) @ RX(-theta_1) @ U
    theta_3 = 2*np.arcsin(1j*RX2[0, 1])

    return theta_1, theta_2, theta_3

And now let's test.

In [64]:
random_seed = 0

# Draw a random unitary.
U = unitary_group.rvs(2, random_state=random_seed) 
# Renormalize the first raw to set det U = 1.
U[1] = U[1] / np.linalg.det(U) 

theta_1, theta_2, theta_3 = XZX_decomposition(U)
check = np.allclose(RX(theta_1) @ RZ(theta_2) @ RX(theta_3), U)
print(f'Does it work? {check}')

Does it work? True


# Decomposing arbitrary single-qutrit gate

## Elementary qutrit gates
Qutrit is an abstration for any three-level quantum system, which is probably less familiar than a qubit. Elementary rotation gates for a qutrit are commonly chosen as standard qubit gates acting only on two of the qutrit levels. We will use
\begin{align}
R^{01}_X(\theta)=\begin{pmatrix}\cos\frac{\theta}2&-i\sin\frac{\theta}2&0\\-i\sin\frac{\theta}2&\cos\frac{\theta}2&0\\0&0&1\end{pmatrix},\quad 
R^{01}_Z(\theta)=\begin{pmatrix}e^{-i\theta/2}&0&0\\0&e^{-i\theta/2}&0\\0&0&1\end{pmatrix}
\end{align}

and

\begin{align}
R^{12}_X(\theta)=\begin{pmatrix}1&0&0\\0&cos\frac{\theta}2&-i\sin\frac{\theta}2\\0&-i\sin\frac{\theta}2&\cos\frac{\theta}2\end{pmatrix},\quad 
R^{12}_Z(\theta)=\begin{pmatrix}1&0&0\\0&e^{-i\theta/2}&0\\0&0&e^{-i\theta/2}\end{pmatrix}
\end{align}

## Is there a an off-the-shelf result? (I'm bad at googling)
A general single-qutrit transformation lives in $SU(3)$ and hence has $3^2-1=8$ real parameters. Hence, we expect to have a decomposition which is a product of 8 elementary rotations. But how exactly will it look, and how to determine the parameters? To my surprise, the only explicit result I found is equation 4 here [https://arxiv.org/abs/1105.5485](https://arxiv.org/abs/1105.5485) which goes something like
$$U = R_Y^{01}R_Y^{02}R_Y^{01}R_Z^{01}R_Z^{02}R_Y^{01}R_Y^{02}R_Y^{01} \ .$$

The result is based on a particular Cartan decomposition of $SU(3)$ (which is non-unique) and does not look very intuitive. Moreover, apparently no explicit algorithm to find the parameters of these rotations (which I ommited in the equation) is given.

## A simple derivation
I will now give another explicit decomposition for a general single-qutrit gate with elementary derivation and explicit procedure to compute the parameters. I begin with separating general $U\in SU(3)$ in blocks
$$U=\begin{pmatrix}A&B\\C&d\end{pmatrix}=\begin{pmatrix} a_{00}& a_{01} & b_0\\ a_{10} & a_{11} & b_1\\ c_0&c_1&d\end{pmatrix}$$.
Now let us write a [singular value decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition)(SVD) for $2\times 2$ matrix $A$
$$A = V \Sigma W^\dagger$$.
Here $V$ and $W$ are unitary while $\Sigma=\operatorname{diag}(\sigma_0,\sigma_1)$ is diagonal with $\sigma_i\ge0$. SVD is a very useful  factorization that works for arbitrary matrices, not necessarily Hermitian/Unitary (or even square). Using SVD of $A$ we can factorize $U$ as follows
$$U=\begin{pmatrix}V&0\\0&1\end{pmatrix}\begin{pmatrix}\Sigma&V^\dagger B\\ CW&D\end{pmatrix}\begin{pmatrix}W^\dagger&0\\0&1\end{pmatrix}=V^{01}\begin{pmatrix}\Sigma&V^\dagger B\\ CW&D\end{pmatrix}\left(W^{01}\right)^\dagger \label{U part}$$
The left and right matrices act only within the first two levels. The middle matrix looks like it acts and all levels of the qutrit, but in fact it is not. I will leave at as an excersize to the reader to show that matrix $A$ necessarily has a unit eigenvalue, we will assume that $\sigma_{0}=1$. In a unitary matrix each row and colums have unit norm. Therefore, if some entry in a unitary matrix is equal to one, all other elements in the corresponding row and column must be zero. Hehnce the middle matrix in \eqref{U part} in fact looks like
$$H^{12}:=\begin{pmatrix}1&0&0\\ 0&\sigma_{1}& b'\\ 0&c'&d\end{pmatrix}$$
and hence only acts on levels 1 and 2. Therefore, the overall decomposition looks like

<img src="myimages/qutrit/qutrit.png" alt="Drawing" style="width: 400px;"/>

Needless to say that the lines on this sketch are not qubits, but individual levels of the qutrit. 

Therefore, we have split a general qutrit gate into three gates involving two levels only. Each of these can be further decomposed into three elementary rotatins using Euler's method for single-qubit gates. It is a bit unfortunate that the overall parameter count is then $3\times 3=9$ instead of the optimal 8. At the moment I am not sure if this is a reduncancy one can eliminate, of decomposition of the type $U^{01}U^{12}U^{01}$ with only 8 parameters does not exists, and one has mix the levels more frequently as in equation \erqef{U3 cart 1}.

## Implementation

Let's go ahead and implement this decomposition. Again, I will not care about efficiency or potential edge cases.

In [93]:
# Embedding qubit gates into qutrit.

def embed(U_qubit, levels):
    U = np.eye(3, dtype=np.complex64)
    if levels == '01':
        U[:2, :2]=U_qubit
    elif levels == '12':
        U[1:, 1:]=U_qubit
    else:
        raise TypeError(f'Levels {levels} not supported.')
        
    return U

def VHW_decomposition(U):
    assert np.allclose(U.conj().T @ U, np.eye(3)), 'Input in not unitary.' 
    assert np.allclose(np.linalg.det(U), 1), 'Input is not special.'
    
    
    A = U[:2, :2] # 2x2 submatrix
    V, S, W = np.linalg.svd(A) # SVD, V and W will act in '01' subspace. 

    H3 = embed(V.conj().T, '01') @ U @ embed(W.conj().T, '01')
    H = H3[1:, 1:] # H3 should act non-trivially only in '12' subspace.

    return V, H, W

Let's verify.

In [94]:
# Draw a random unitary and normalize so that det U=1.
random_seed=0
U = unitary_group.rvs(3, random_state=random_seed)
U[0] = U[0]/np.linalg.det(U)

# And now we check.
V, H, W = VHW_decomposition(U)
check = np.allclose(embed(V, '01') @ embed(H, '12') @ embed(W, '01'), U)
print(f'Does it work?: {check}')

Does it work?: True


In [97]:
np.linalg.det(W)

(-0.13759636240745646+0.9904883851172793j)

Of course, you can also decompose each of the 2-level blocks using XZX decomposition. For completeness, I'll give the relevant function.

In [92]:
def XZX_qutrit_decompose(U):
    V, H, W = VHW_decomposition(U)
    return XZX_decomposition(V) + XZX_decomposition(H) + XZX_decomposition(W)

# Draw.
random_seed=0
U = unitary_group.rvs(3, random_state=random_seed)
U[0] = U[0]/np.linalg.det(U)

# Check
theta = XZX_qutrit_decompose(U)
V01 = embed(RX(theta[0]), '01') @ embed(RZ(theta[1]), '01') @ embed(RX(theta[2]), '01')
H12 = embed(RX(theta[3]), '12') @ embed(RZ(theta[4]), '12') @ embed(RX(theta[5]), '12')
W01 = embed(RX(theta[6]), '01') @ embed(RZ(theta[7]), '01') @ embed(RX(theta[8]), '01')

check = np.allclose(V01 @ H12 @ W01, U)
print(f'Does it work?: {check}')

AssertionError: Input matrix has non-unit determinant.